# <span style ='color:purple'>Import & settings

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
%reload_ext autoreload
%autoreload 2

from data_provider.data_factory import data_provider
from tabular.dl_logs import logs_summary, exp_summary

In [3]:
from types import SimpleNamespace
lt_logs_df = pd.read_csv(os.path.join('./logs/long_term_forecast.csv'), index_col = 0)
usc_args = SimpleNamespace(**lt_logs_df.loc[207].to_dict())

In [4]:
seq_len = 32
pred_len = seq_len
usc_args.seq_len = seq_len
usc_args.pred_len = pred_len
usc_args.scale = False

# <span style ='color:cyan'> Data loading

In [5]:
uscds, uscdl = data_provider(args= usc_args, flag = 'train')

train 19749
